In [1]:
import pandas as pd
from selenium import webdriver
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json
from coordenadas import pasando_a_string
from coordenadas import get_coordenadas
from coordenadas import rellenando_datasetfsq
import foursquare
from config import dic_categorias
from pymongo import MongoClient

In [2]:
load_dotenv()

True

In [3]:
tokg = os.getenv("tokg")
tokf3 = os.getenv ("tokf3")
client2= MongoClient ("localhost:27017")

In [354]:
client = foursquare.Foursquare( access_token= tokf3 )
gmaps = googlemaps.Client(key= tokg )
db = client2.get_database("ciudades")

# he borrado todo lo que tenia de datasets, y empiezo directamente con lo que sería el flujo de la api

input: ciudad y pais

In [53]:
dallas="-23.5505199, -46.633"

In [67]:
pruebadallas=client.venues.search(params={'query': 'hotel', 'll': "32.7766642,-96.79698789999999", "radius": 2000, "limit": 50})

In [391]:
gmaps.geocode( address= "toledo")

[{'address_components': [{'long_name': 'Toledo',
    'short_name': 'Toledo',
    'types': ['locality', 'political']},
   {'long_name': 'Lucas County',
    'short_name': 'Lucas County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Ohio',
    'short_name': 'OH',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']}],
  'formatted_address': 'Toledo, OH, USA',
  'geometry': {'bounds': {'northeast': {'lat': 41.732844, 'lng': -83.454229},
    'southwest': {'lat': 41.580266, 'lng': -83.6942832}},
   'location': {'lat': 41.6528052, 'lng': -83.5378674},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 41.732844, 'lng': -83.454229},
    'southwest': {'lat': 41.580266, 'lng': -83.6942832}}},
  'place_id': 'ChIJeU4e_C2HO4gRRcM6RZ_IPHw',
  'types': ['locality', 'political']}]

In [392]:
gmaps.geocode( address= "toledo, españa")

[{'address_components': [{'long_name': 'Toledo',
    'short_name': 'Toledo',
    'types': ['locality', 'political']},
   {'long_name': 'Toledo',
    'short_name': 'TO',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Castile-La Mancha',
    'short_name': 'CM',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Spain',
    'short_name': 'ES',
    'types': ['country', 'political']}],
  'formatted_address': 'Toledo, Spain',
  'geometry': {'bounds': {'northeast': {'lat': 39.88605099999999,
     'lng': -3.9192423},
    'southwest': {'lat': 39.8383676, 'lng': -4.0796176}},
   'location': {'lat': 39.8628316, 'lng': -4.027323099999999},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 39.88605099999999, 'lng': -3.9192423},
    'southwest': {'lat': 39.8383676, 'lng': -4.0796176}}},
  'place_id': 'ChIJ8f21C60Lag0R_q11auhbf8Y',
  'types': ['locality', 'political']}]

In [336]:
a=db.list_collection_names()

In [389]:
test_and_create ("dallas", "usa")

In [108]:
def test (c):
    

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ciudades'), '-23.5505199, -46.633')

In [388]:
def test_and_create( ciudad, pais):
    try:   
            busqueda= ciudad + ", " + pais #unimos en un string para pasarselo como búsqueda a gplaces
            coords=gmaps.geocode( address= busqueda)
            nombre = coords[0]['address_components'][0]['short_name'] # nos quedamos con el nombre, que será el de la colección
            a=coords[0]["geometry"]["location"]["lat"] 
            b=coords[0]["geometry"]["location"]["lng"]
            c = str (a) +"," + str(b) # Formato que luego entienda foursquare
            dic= {"nombre": nombre,
                 "coordenadas": c}
            test (c)
            
            db.create_collection (nombre)
        
    except: "los datos introducidos no son correctos"
        
    return  get_coords (nombre, dic)

In [208]:
def get_coords(nombre, dic):
    collection= nombre 
    db[collection].insert_one (dic)
    coord= dic["coordenadas"]
    return get_data_fields (coord, collection)
        

In [ ]:
def get_data_fields (coord, collection):
    tokf3 = os.getenv ("tokf3")
    client = foursquare.Foursquare( access_token= tokf3 )
    cantidad=[]
    dic_insert={}
    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        cantidad= len (lista)
        dice1={categoria: {"número":cantidad }}
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            direccion= lista[i]["location"]["formattedAddress"]
        
            dice2 = {"nombre": nombre, 
               "latitud":latitud,
               "longitud": longitud,
               "direccion":direccion }
            dice.append (dice2)          
        
        dice1[categoria].update ({"listado": dice})
    db["collection"].insert_one (dice1)
    
    return query

In [150]:
db.list_collection_names()

['Colonia',
 'Washington D. C',
 'ciudad',
 'Nueva York',
 'Osaka',
 'Londres',
 'Ciudad de México',
 'Houston',
 'Tokio',
 'Chicago',
 '#16',
 'Los Ángeles',
 'Seúl',
 'Dallas',
 'París',
 'Pekín',
 'Shanghái',
 'Moscú']